# utils-qgofer

> Utility functions that cut across most qgofer projects

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
!pip install orjson

In [ ]:
#| hide
!pip install requests

In [ ]:
#| export
import os
import base64
from pathlib import Path
from typing import Optional
import logging
import orjson
import requests
import shutil
from typing import Dict, Any
from __future__ import annotations
import uuid
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import datetime
import sqlite3
import sys

In [ ]:
#| export
def get_env_var(var_name: str, default=None):
    """
    Returns the value of an environment variable if it exists,
    otherwise returns the default value.
    """
    try:
        return os.environ[var_name]
    except KeyError:
        return default

In [ ]:
#| export
def generate_uuid():
    """Generates a uuid 4 string
    Returns:
        an ascii friendly uuid4 string.
    """
    return base64.urlsafe_b64encode(uuid.uuid4().bytes).rstrip(b"=").decode("ascii")

In [ ]:
#| export
def init_path(path: Optional[Path] = None) -> Path:
    """Convert the string to a path object."""
    if path is None:
        return Path.home()
    return Path(path).expanduser().resolve()

In [ ]:
#| export
def get_log_path(path: Optional[Path] = None) -> Path:
    """Returns the log path."""
    if path is None:
        path = init_path(path) / '.qgofer' / 'logs'
        path.mkdir(parents=True, exist_ok=True)
    return Path(path).expanduser().resolve()

In [ ]:
#| export
def get_logger(log_path=None, run_id=None) -> logging.Logger:
    """Returns a generic logger for logging relevant information pertaining to run of qgofer.

    Returns:
        Python logger object
    """
    log_path = get_log_path(log_path)
    logger = logging.getLogger("qgofer")
    logger.setLevel(logging.DEBUG)

    if not run_id:
        run_id = generate_uuid()

    file_path = f"{log_path}/{run_id}_log_{datetime.datetime.now().strftime('%y_%m_%d_%H_%M_%S')}.log"
    if not logger.handlers:
        # create file handler which logs even debug messages
        fh = logging.FileHandler(file_path)
        fh.setLevel(logging.DEBUG)
        # create console handler with a higher log level
        ch = logging.StreamHandler(stream=sys.stdout)
        ch.setLevel(logging.DEBUG)

        # create formatter and add it to the handlers
        formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)

        # add the handlers to the logger
        logger.addHandler(fh)
        logger.addHandler(ch)
        logger.info(f"Logging to {file_path}")

    return logger

In [ ]:
#| export
def get_failed_response(
    error_message="Some kind of API error ccured while interacting with the given URL",
) -> requests.Response:
    failed_response = requests.Response()
    failed_response.status_code = 500
    failed_response.reason = error_message
    failed_response._content = orjson.dumps({"message": f"{error_message}"})
    return failed_response


async def make_api_request(
    url: str,
    http_method: str = "GET",
    headers: dict = {},
    data: dict = {},
    auth: tuple = (),
    cookies: dict = {},
    params: dict = {},
) -> requests.Response:
    """Makes an API request to the given url with the given parameters."""
    if not all(headers.values()):
        return get_failed_response()
    s = requests.Session()
    retries = Retry(
        total=10,
        backoff_factor=0.1,
        status_forcelist=[403, 406, 408, 413, 429, 500, 502, 503, 504],
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://", HTTPAdapter(max_retries=retries))

    try:
        req = requests.Request(
            http_method,
            url,
            data=data,
            headers=headers,
            auth=auth,
            cookies=cookies,
            params=params,
        )
        prepped = req.prepare()
        resp = s.send(prepped)
        return resp
    except Exception as e:
        get_logger().error("Connection error while fetching data {}".format(e))
        return get_failed_response()


In [ ]:
#| export

class QGoferDBWrapper:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def create_table(self, table_name, columns):
        """
        Create a table in the database
        :param table_name: name of the table to create
        :param columns: list of columns to create wih their data type
        :return: None
        """
        self.cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns})")
        self.conn.commit()

    def insert(self, table_name, columns, values):
        """
        Insert a row into the database
        :param table_name: name of the table to insert into
        :param columns: list of columns to insert into
        :param values: list of values to insert into the table
        :return: None
        """
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
        print(sql)
        self.cursor.execute(sql)
        self.conn.commit()

    def insert_many(self, table_name, columns, values):
        """
        Insert multiple rows into the database
        :param table_name: name of the table to insert into
        :param columns: list of columns to insert into
        :param values: list of values to insert into the table
        :return: None
        """
        self.cursor.executemany(f"INSERT INTO {table_name} ({columns}) VALUES ({values})", values)
        self.conn.commit()

    def select(self, table_name, columns, where=None):
        """
        Select rows from the database
        :param table_name: name of the table to select from
        :param columns: list of columns to select
        :param where: where clause to filter rows
        :return: list of rows
        """
        if where is None:
            self.cursor.execute(f"SELECT ({columns}) FROM {table_name}")
        else:
            self.cursor.execute(f"SELECT ({columns}) FROM {table_name} WHERE {where}")
        return self.cursor.fetchall()

    def search(self, table_name, column_name, q):
        """
        Search rows from the database
        :param table_name: name of the table to search from
        :param column_name: name of the column to search
        :param q: search query
        :return: list of rows
        """
        self.cursor.execute(f"SELECT * FROM {table_name} WHERE {column_name} LIKE '%{q}%'")
        return self.cursor.fetchall()

    def update(self, table_name, set_clause, where):
        """
        Update rows in the database
        :param table_name: name of the table to update
        :param set_clause: set clause to update rows
        :param where: where clause to filter rows
        :return: None
        """
        self.cursor.execute(f"UPDATE {table_name} SET {set_clause} WHERE {where}")
        self.conn.commit()

    def delete(self, table_name, where):
        """
        Delete rows from the database
        :param table_name: name of the table to delete from
        :param where: where clause to filter rows
        :return: None
        """
        self.cursor.execute(f"DELETE FROM {table_name} WHERE {where}")
        self.conn.commit()

    def drop_table(self, table_name):
        """
        Drop a table from the database
        :param table_name: name of the table to drop
        :return: None
        """
        self.cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
        self.conn.commit()



In [ ]:
#| export


log_path = get_log_path()
logger = get_logger(log_path=log_path)


class QGoferConfig:
    """The main appliation class for managing the entirety of qgofer.

    Attributes:
        home: The home directory of the user, defaults to Path.home()
        root_dir: The root directory to index, defaults to Path.home()
    """

    __slots__ = (
        "_home",
        "_root_dir",
        "_qgofer",
        "_qgofer_config",
        "_qgofer_cache",
        "_qgofer_cache_db",
        "_qgofer_logs",
    )
    _instances: Dict[Any, Any] = {}

    def __new__(cls, home, root_dir) -> QGoferConfig:
        """Create a new instance of the class. by using the singleton pattern."""
        if cls not in cls._instances:
            cls._instances[cls] = super(App, cls).__new__(cls)
        return cls._instances[cls]

    def __init__(self, home: Path = Path.home(), root_dir: Path = Path.home()):
        """Initialize the class. using home and root_dir as the home and root directory of qgofer."""
        self._home = home
        self._root_dir = root_dir
        self._qgofer = self._home / '.qgofer'
        self._qgofer.mkdir(parents=True, exist_ok=True)
        self._qgofer_config = self._qgofer / 'config.toml'
        self._qgofer_config.touch(exist_ok=True)
        self._qgofer_cache = self._qgofer / 'cache'
        self._qgofer_cache.mkdir(parents=True, exist_ok=True)
        self._qgofer_cache_db = self._qgofer_cache / 'qgofer.db'
        self._qgofer_cache_db.touch(exist_ok=True)
        self._qgofer_logs = self._qgofer / 'logs'
        self._qgofer_logs.mkdir(parents=True, exist_ok=True)

    @property
    def home(self) -> Path:
        """Return the home directory of the user."""
        return self._home

    @property
    def qgofer(self) -> Path:
        """Return the qgofer directory."""
        return self._qgofer

    @property
    def qgofer_config(self) -> Path:
        """Return the qgofer config file."""
        return self._qgofer_config

    @property
    def qgofer_cache(self) -> Path:
        """Return the qgofer cache directory."""
        return self._qgofer_cache

    @property
    def qgofer_cache_db(self) -> Path:
        """Return the qgofer cache database."""
        return self._qgofer_cache_db

    def _clear_cache_db(self) -> None:
        """Clear the qgofer cache database."""
        qgofer_db_wrapper = QGoferDBWrapper(self.qgofer_cache_db)
        try:
            os.remove(self.qgofer_cache_db)
        except FileNotFoundError as err:
            logger.error(err)

    def clear_cache(self) -> None:
        """Clears the cache folder"""
        try:
            shutil.rmtree(self._qgofer_cache)
        except OSError as err:
            logger.error(err)
        self._clear_cache_db()

    def clear_logs(self) -> None:
        """This function clears the logs folder
        """
        try:
            shutil.rmtree(self._qgofer_logs)
            self._qgofer_logs.mkdir(parents=True, exist_ok=True)
        except OSError as err:
            logger.error(err)

    def __repr__(self) -> str:
        """Return the representation of the class."""
        return f"{self.__class__.__name__}({self.home})"

    def __str__(self) -> str:
        """Return the string representation of the class."""
        return "Qgofer your personal assistant"

2024-03-02 13:56:30,166 - qgofer - INFO - Logging to C:\Users\acqua\.qgofer\logs/13HpRU94QkyQLjSip7dEdQ_log_24_03_02_13_56_30.log


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()